In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [52]:
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import textwrap

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("model/")

In [5]:
bbc_unprocess = pd.read_csv('PPT/bbc_unprocess.csv', names=[0,1], header=0)
bbc_unprocess[1] = bbc_unprocess[1].fillna("")
nytimes_unprocess = pd.read_csv('PPT/nytimes_unprocess.csv', names=[0,1], header=0)
nytimes_unprocess[1] = nytimes_unprocess[1].fillna("")

bbc_unprocess_agg = bbc_unprocess.groupby(bbc_unprocess.index // 3)[1].agg(lambda x: x.iloc[0]+ " \n " + x.iloc[1] + " \n " + x.iloc[2]).to_frame()
bbc_unprocess_agg_time = bbc_unprocess.groupby(bbc_unprocess.index // 3)[0].agg(lambda x: x.iloc[0]).to_frame()
bbc_unprocess_agg_df = pd.concat([bbc_unprocess_agg_time, bbc_unprocess_agg], axis=1)
bbc_unprocess_agg_df[2] = "BBC"

nytimes_unprocess_agg = nytimes_unprocess.groupby(nytimes_unprocess.index // 3)[1].agg(lambda x: x.iloc[0]+ " \n " + x.iloc[1] + " \n " + x.iloc[2]).to_frame()
nytimes_unprocess_agg_time = nytimes_unprocess.groupby(nytimes_unprocess.index // 3)[0].agg(lambda x: x.iloc[0]).to_frame()
nytimes_unprocess_agg_df = pd.concat([nytimes_unprocess_agg_time, nytimes_unprocess_agg], axis=1)
nytimes_unprocess_agg_df[2] = "NYTimes"

data_unprocess_agg_df = pd.concat([bbc_unprocess_agg_df, nytimes_unprocess_agg_df]).sort_values(0)
data_unprocess_agg_df[3] = 0

In [49]:
# inputs = tokenizer("I am so sorry", return_tensors="pt")
# with torch.no_grad():
#     print(model(**inputs).logits.tolist()[0])

[-0.39768850803375244, -0.5224173665046692, -1.4474537372589111, -1.5706732273101807, -2.1861884593963623]


In [58]:
# for index, (i, line) in tqdm(enumerate(data_unprocess_agg_df.iterrows()), total=data_unprocess_agg_df.shape[0]):
#   with torch.no_grad():
#     try:
#       line_token = tokenizer(line[1], return_tensors="pt")
#       result = model(**line_token).logits.tolist()[0]
#     except RuntimeError:
#       items = textwrap.wrap(line[1], 1000)
#       result_temp = []
#       for item in items:
#         item_token = tokenizer(item, return_tensors="pt")
#         result_temp.append(model(**item_token).logits.tolist()[0])
#       result = np.mean(result_temp, axis=0)
#   with open("PPT/sentiment_scores_tunning.txt", "a") as file:
#     file.write(str(index) + " " + ",".join([str(i) for i in result]) + "\n")

100%|██████████| 16030/16030 [3:14:46<00:00,  1.37it/s]


In [29]:
# with open("PPT/sentiment_scores_tunning.txt", "r") as file:
#   lines = file.readlines()

# ans = []
# for line in lines:
#    line = [float(i) for i in line.strip().split()[1].split(",")]
#    line = [(float(i)-min(line))/(max(line)-min(line)) for i in line]
#    line = [i/sum(line) for i in line]
#    ans.append(sum(i * j for i, j in zip(np.arange(1,6), line)))
# data_unprocess_agg_df[3] = ans
# # data_unprocess_agg_df.to_csv('PPT/data_unprocess_agg_df_sentiment_tuned.csv', index=False)

In [105]:
from transformers import AutoTokenizer
from transformers import BertConfig
from sklearn.metrics.pairwise import cosine_similarity

tokenizer_tuned = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
config_tuned = BertConfig.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", output_hidden_states=True)
model_tuned = AutoModelForSequenceClassification.from_pretrained("model/", config=config_tuned)

def get_hidden_states(encoded, token_ids_word, model):
  with torch.no_grad():
    output = model(**encoded)
  states = output.hidden_states
  output = torch.stack([states[i] for i in [0]]).sum(0).squeeze()
  word_tokens_output = output[token_ids_word]
  return word_tokens_output.mean(dim=0).reshape(1, -1)

def get_word_vector(sent, tokenizer=tokenizer_tuned, model=model_tuned):
  encoded = tokenizer.encode_plus(sent, return_tensors="pt")
  token_ids_word = np.where(np.array(encoded.word_ids()) == 0)
  return get_hidden_states(encoded, token_ids_word, model)

def get_sencentence_vector(sent):
  sents = sent.split()
  if len(sents) > 1:
    return np.mean([get_word_vector(i) for i in sents], axis=0)
  else:
    return get_word_vector(sent)

word_embedding1 = get_sencentence_vector("his cookies")
word_embedding2 = get_sencentence_vector("cookies")


In [113]:
np.mean([get_word_vector("his"), get_word_vector("coockied")], axis=0).shape

(1, 768)